In [37]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime
from dateutil.relativedelta import relativedelta

def run_taa_momentum_strategy_precise(tickers, start, end, initial_balance):
    # 날짜 설정
    start_dt = pd.to_datetime(start)
    end_dt = pd.to_datetime(end)
    start_download = start_dt - relativedelta(months=13)  # 12개월 룩백 + 1개월 여유

    # 1. 데이터 다운로드 (일일 데이터)
    data = yf.download(tickers, start=start_download.strftime('%Y-%m-%d'), end=end_dt.strftime('%Y-%m-%d'), progress=False, auto_adjust=False)['Adj Close']
    data = data.dropna(how='all')

    # 2. 월별 마지막 거래일 찾기
    # rebalance_dates = data.resample('M').apply(lambda x: x.index[-1]).tolist()
    rebalance_dates = data.index.to_series().groupby(data.index.to_period("M")).last().tolist()

    # 결과 저장용
    portfolio_returns = []
    dates = []
    trades_log = []

    for i in range(12, len(rebalance_dates)):
        date = rebalance_dates[i]
        past_data = data[:date]

        momentum_scores = pd.Series(index=tickers, dtype=float)

        # 수익률 계산 (일일 기준으로 정확히 계산)
        for ticker in tickers:
            try:
                p1 = past_data[ticker].iloc[-1]
                p3 = past_data[ticker].loc[:date - relativedelta(months=3)].iloc[-1]
                p6 = past_data[ticker].loc[:date - relativedelta(months=6)].iloc[-1]
                p12 = past_data[ticker].loc[:date - relativedelta(months=12)].iloc[-1]
                r1 = (p1 / p1) - 1
                r3 = (p1 / p3) - 1
                r6 = (p1 / p6) - 1
                r12 = (p1 / p12) - 1
                score = 0.25 * r3 + 0.25 * r6 + 0.25 * r12 + 0.25 * r1
                momentum_scores[ticker] = score
            except:
                momentum_scores[ticker] = np.nan

        top_assets = momentum_scores.dropna().sort_values(ascending=False).head(3).index.tolist()

        # 10개월 이동평균 (약 200거래일)
        ma_200 = data[tickers].rolling(window=200).mean()
        valid_assets = []
        for asset in top_assets:
            price = data.loc[date, asset]
            ma = ma_200.loc[date, asset]
            if price > ma:
                valid_assets.append(asset)

        # 최종 포트폴리오 구성
        weights = {}
        if valid_assets:
            w = 1 / len(valid_assets)
            for asset in valid_assets:
                weights[asset] = w
        else:
            weights['CASH'] = 1.0

        # 수익률 계산 (다음 리밸런싱 날짜까지 수익률)
        if i + 1 < len(rebalance_dates):
            next_date = rebalance_dates[i + 1]
        else:
            next_date = data.index[-1]

        ret = 0
        for asset, w in weights.items():
            if asset == 'CASH':
                ret += 0
            else:
                try:
                    start_price = data.loc[date, asset]
                    end_price = data.loc[next_date, asset]
                    asset_ret = (end_price / start_price) - 1
                    ret += w * asset_ret
                except:
                    ret += 0

        portfolio_returns.append(ret)
        dates.append(next_date)

        # 거래 로그
        trade_info = {
            'Trade Date': date.strftime('%Y-%m-%d'),
            'Start Date': (date + pd.offsets.BDay(1)).strftime('%Y-%m-%d'),
            'End Date': next_date.strftime('%Y-%m-%d'),
            'Selected ETFs': ', '.join(top_assets),
            'Final Holdings': ', '.join([f"{asset} ({weights[asset]*100:.2f}%)" if asset != 'CASH' else 'CASH (100%)' for asset in weights]),
            'Momentum Scores': ', '.join([f"{etf}: {momentum_scores[etf]*100:.2f}%" for etf in top_assets])
        }
        trades_log.append(trade_info)

    # 수익률 & 잔고 계산
    portfolio_returns_df = pd.DataFrame(portfolio_returns, index=dates, columns=['Portfolio Return'])
    portfolio_returns_df['Portfolio Return'] = (portfolio_returns_df['Portfolio Return'] * 100).round(2)
    portfolio_returns_df['Portfolio Balance'] = (1 + pd.Series(portfolio_returns, index=dates)).cumprod() * initial_balance

    # Performance Summary
    final_balance = portfolio_returns_df['Portfolio Balance'].iloc[-1]
    cagr = (final_balance / initial_balance) ** (1 / (len(portfolio_returns_df) / 12)) - 1
    std_dev = (portfolio_returns_df['Portfolio Return'] / 100).std() * np.sqrt(12)
    sharpe = cagr / std_dev if std_dev != 0 else np.nan
    downside_std = (portfolio_returns_df['Portfolio Return'] / 100)[(portfolio_returns_df['Portfolio Return'] < 0)].std() * np.sqrt(12)
    sortino = cagr / downside_std if downside_std != 0 else np.nan
    max_drawdown = ((portfolio_returns_df['Portfolio Balance'] / portfolio_returns_df['Portfolio Balance'].cummax()) - 1).min()

    performance_summary = pd.DataFrame({
        'Metric': [
            'Start Balance', 'End Balance', 'Annualized Return (CAGR)',
            'Standard Deviation', 'Maximum Drawdown', 'Sharpe Ratio', 'Sortino Ratio'
        ],
        'Value': [
            f"${initial_balance:,.2f}", f"${final_balance:,.2f}",
            f"{cagr*100:.2f}%", f"{std_dev*100:.2f}%",
            f"{max_drawdown*100:.2f}%", f"{sharpe:.2f}", f"{sortino:.2f}"
        ]
    })

    trades_df = pd.DataFrame(trades_log)

    return portfolio_returns_df, performance_summary, trades_df


tickers = ['SPY', 'IWM']
start = '2016-01-01'
end = '2025-02-28'
initial_balance = 10000

returns_df, summary_df, trades_df = run_taa_momentum_strategy_precise(tickers, start, end, initial_balance)

# print(summary_df)
display(trades_df.head())
# returns_df.plot(title="Precise TAA Portfolio Performance", y='Portfolio Balance')


,Trade Date,Start Date,End Date,Selected ETFs,Final Holdings,Momentum Scores
0,2015-12-31,2016-01-01,2016-01-29,"SPY, IWM",CASH (100%),"SPY: 2.10%, IWM: -2.42%"
1,2016-01-29,2016-02-01,2016-02-29,"SPY, IWM",CASH (100%),"SPY: -3.97%, IWM: -9.35%"
2,2016-02-29,2016-03-01,2016-03-31,"SPY, IWM",CASH (100%),"SPY: -3.77%, IWM: -9.81%"
3,2016-03-31,2016-04-01,2016-04-29,"SPY, IWM",SPY (100.00%),"SPY: 2.86%, IWM: -2.29%"
4,2016-04-29,2016-05-02,2016-05-31,"SPY, IWM","SPY (50.00%), IWM (50.00%)","SPY: 1.76%, IWM: -0.12%"
